In [1]:
from cosecha_colectiva import miscelaneous as m
import pandas as pd
m.try_find_conf_file()


from cosecha_colectiva import main_tester as mt
from cosecha_colectiva import config
from cosecha_colectiva import query_cosecha as qc
from cosecha_colectiva import test_cosecha_api as tca
from cosecha_colectiva import CAF_DB_Monitor as CDM

from cosecha_colectiva import miscelaneous as m

import logging
import pathlib
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


[2025-05-07 17:48:58,925][INFO]: Connecting root@junction.proxy.rlwy.net:19994
[2025-05-07 17:49:00,135][INFO]: Connected root@junction.proxy.rlwy.net:19994


In [2]:
now = datetime.now()
log_date = now.strftime("%Y-%m-%d_%H_%M_%S")
log_file = pathlib.Path(config.log_dir, log_date+'CreacionGrupo.log')
logging.basicConfig(filename=log_file, encoding='utf-8', level=logging.INFO)

In [3]:
xls_name = 'Verde_Futuro_Test.xlsx'
xls_name = 'Verde_Futuro.xlsx'

xls_name = 'Tropa Gusgones_2.0.xlsx'
xls_name = 'Tropa Gusgones_2.0_Test.xlsx'

xls_name = 'Alcancía Viva.xlsx'
#xls_name = 'Alcancía Viva Test.xlsx'

xls_name = 'Licenciados_Asociados.xlsx'
#xls_name = 'Licenciados_Asociados_Test.xlsx'

xls_name = 'Juntas_ahorramos.xlsx'
xls_name = 'Juntes_ahorramos.xlsx'


xls_name = 'Procoseq.xlsx'
#xls_name = 'Juntos_ahorramos.xlsx'



#xls_name = 'CAF_GrupoTest.xlsx'
#xls_name = 'CAF_GrupoHidalgo.xlsx'
#xls_name = 'GrupoCafam.xlsx'

#xls_name = 'Yeiyei'

In [4]:
xls_name

'Procoseq.xlsx'

In [5]:
[id_grupo, grupo_found] = qc.get_grupo_id_by_name(xls_name.replace(".xlsx", ""))
print(id_grupo)
#qc.delete_grupo(id_grupo, solo_sesiones=True)
#qc.delete_grupo(id_grupo)

2618


In [6]:
if not grupo_found:
    grupo_data = m.read_caf_excel_sheet(pathlib.Path(xls_name), 'Grupo')
    id_grupo = qc.insert_grupo(grupo_data.to_dict('records'))
    id_grupo = id_grupo['Grupo_id']
    
id_grupo

2618

In [7]:
if not grupo_found:

    socios_xls = pd.DataFrame(m.get_dict_usuarios_xls(xls_name))
    # Solo crea socios cuando viene los datos completos
    if not socios_xls['Fecha_nac'].isna().all():
        print('Creando usuarios')
        tca.CAF_API_group_creation_tester.create_users(xls_name)

    grupo_socio_df = qc.prepare_insert_socio_grupo(xls_name, id_grupo)

    df_acuerdos = m.read_caf_excel_sheet(pathlib.Path(xls_name), "Acuerdos")
    df_acuerdos['Grupo_id'] = id_grupo
    df_acuerdos['Fecha_acuerdos'] = datetime.strftime(date.today(), format='%Y-%m-%d')
    df_acuerdos['Status'] = 1

    df_acuerdos['Id_socio_administrador'] = grupo_socio_df.loc[0,'Socio_id']
    df_acuerdos['Id_socio_administrador_suplente'] = grupo_socio_df.loc[1,'Socio_id']
    
    qc.insert_acuerdos(df_acuerdos.to_dict('records')[0])

    
    


In [8]:
qc.delete_grupo(id_grupo, solo_sesiones=True, force_delete=True, desde_sesion_0=True)


#qc.restart_grupo_acciones(id_grupo)

([], 'transaccion_prestamos')
([], 'transacciones')
([], 'interes_prestamo')
([], 'ganancias')
([], 'multas')
([], 'prestamos')
([], 'prestamos')
([{'Asistencia_id': 138141}, {'Asistencia_id': 138142}, {'Asistencia_id': 138143}, {'Asistencia_id': 138144}, {'Asistencia_id': 138145}, {'Asistencia_id': 138146}, {'Asistencia_id': 138147}, {'Asistencia_id': 138148}], 'asistencias')
DELETE from railway.asistencias WHERE Asistencia_id in (138141, 138142, 138143, 138144, 138145, 138146, 138147, 138148)
([{'Sesion_id': 19805}, {'Sesion_id': 19819}], 'sesiones')
DELETE from railway.sesiones WHERE Sesion_id in (19805, 19819)
([], 'acuerdos')


In [9]:
start_month = 1
end_month = 1
initial_date = (date.today() - relativedelta(months=end_month))

monitor_grupo = CDM.CAF_DB_Monitor(id_grupo, xls_name, starting_date=initial_date)
monitor_grupo.socios_acciones


,Grupo_socio_id,Socio_id,Tipo_socio,Status,Acciones,Grupo_id,CURP,xls_order
0,9091,3037,ADMIN,1,0,2618,SAML830823HQTNRS06,0
1,9092,3039,SOCIO,1,0,2618,SOHJ780208HQTTRS06,1
2,9093,3040,SUPLENTE,1,0,2618,AEPJ500308HQTNRL01,2
3,9094,3042,SOCIO,1,0,2618,MAHA830218HQTRRD01,3
4,9095,3038,SOCIO,1,0,2618,MAPC560916HQTRNP02,4
5,9096,3041,SOCIO,1,0,2618,MERC531203MQTNSL09,5
6,9097,3043,SOCIO,1,0,2618,MUOJ920730HQTXLL01,6
7,9098,3044,SOCIO,1,0,2618,OEER620207HQTLSB00,7


In [10]:

for month in range(start_month, end_month+1):

    print('month: ', month)

    try:
        monitor_grupo.actualiza_todo_sesion(xls_name, month, type_xls='NEW', bd_update=True)
    except Exception as e:
        print(e)
        raise(e)
        break

month:  1


In [11]:
monitor_grupo.socios_acciones

,Grupo_socio_id,Socio_id,Grupo_id,Tipo_socio,Acciones,Status,CURP
0,9091,3037,2618,ADMIN,100,1,SAML830823HQTNRS06
1,9092,3039,2618,SOCIO,100,1,SOHJ780208HQTTRS06
2,9093,3040,2618,SUPLENTE,100,1,AEPJ500308HQTNRL01
3,9094,3042,2618,SOCIO,100,1,MAHA830218HQTRRD01
4,9095,3038,2618,SOCIO,100,1,MAPC560916HQTRNP02
5,9096,3041,2618,SOCIO,100,1,MERC531203MQTNSL09
6,9097,3043,2618,SOCIO,100,1,MUOJ920730HQTXLL01
7,9098,3044,2618,SOCIO,100,1,OEER620207HQTLSB00


In [12]:
monitor_grupo.prestamos

,Prestamo_id,Monto_prestamo,Monto_pagado,Interes_generado,Interes_pagado,Fecha_inicial,Fecha_final,Estatus_ampliacion,Observaciones,Num_sesiones,...,Ampliacion_prestamos,Interes_ampliacion,Mod_calculo_interes,Tasa_interes_prestamo_grande,interes_futuro,Ultimo_interes_pagado,debe_interes,sobrante_abono,Ultimo_abono,Status_socio
0,6590,200.0,0,0,0,2025-04-07,2025-06-07,False,Préstamo generado en script automático,2,...,1,0.0,1,1.5,2.0,0,0,0,0,1


In [13]:
monitor_grupo.sesiones_bd

,Sesion_id,Fecha,Activa,Caja,Acciones,Ganancias,Fecha_prox_reunion,Lugar_prox_reunion,Tipo_sesion,Grupo_id,created_at
0,19820,2025-04-07,0,600.0,800,0.0,2025-05-07,Sesion creada en script,1,2618,2025-04-07


In [14]:
monitor_grupo.caja

600.0

In [15]:
monitor_grupo.ganancias

,Ganancias_id,Ganancia_accion,Monto_ganancia,Entregada,Socio_id,Sesion_id,periodo
0,80108,0.0,0.0,0,3037,19820,nuevo
1,80109,0.0,0.0,0,3039,19820,nuevo
2,80110,0.0,0.0,0,3040,19820,nuevo
3,80111,0.0,0.0,0,3042,19820,nuevo
4,80112,0.0,0.0,0,3038,19820,nuevo
5,80113,0.0,0.0,0,3041,19820,nuevo
6,80114,0.0,0.0,0,3043,19820,nuevo
7,80115,0.0,0.0,0,3044,19820,nuevo


In [16]:
monitor_grupo.ganancias_acum

,Monto_ganancia
Socio_id,
3037,0.0
3038,0.0
3039,0.0
3040,0.0
3041,0.0
3042,0.0
3043,0.0
3044,0.0


In [17]:
monitor_grupo.socios_acciones

,Grupo_socio_id,Socio_id,Grupo_id,Tipo_socio,Acciones,Status,CURP
0,9091,3037,2618,ADMIN,100,1,SAML830823HQTNRS06
1,9092,3039,2618,SOCIO,100,1,SOHJ780208HQTTRS06
2,9093,3040,2618,SUPLENTE,100,1,AEPJ500308HQTNRL01
3,9094,3042,2618,SOCIO,100,1,MAHA830218HQTRRD01
4,9095,3038,2618,SOCIO,100,1,MAPC560916HQTRNP02
5,9096,3041,2618,SOCIO,100,1,MERC531203MQTNSL09
6,9097,3043,2618,SOCIO,100,1,MUOJ920730HQTXLL01
7,9098,3044,2618,SOCIO,100,1,OEER620207HQTLSB00
